# <font color="red">**3. Correlation-Based FS - Fine-tuning Hyperparameters**</font>

**Author:** Osmar Bolivar

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go

## **1. Monthly data**

In [22]:
#raw = pd.read_csv("./Data/CORR_DATASET.csv", index_col=0)
raw = pd.read_csv("./Data/PC_DATASET.csv", index_col=0)
raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5901 entries, 2010-01-01 to 2024-10-31
Data columns (total 87 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ipc_all  178 non-null    float64
 1   lag_1    5901 non-null   float64
 2   lag_2    5901 non-null   float64
 3   lag_3    5901 non-null   float64
 4   lag_6    5901 non-null   float64
 5   lag_9    5901 non-null   float64
 6   lag_12   5901 non-null   float64
 7   freq     5901 non-null   object 
 8   PC_1     5901 non-null   float64
 9   PC_2     5901 non-null   float64
 10  PC_3     5901 non-null   float64
 11  PC_4     5901 non-null   float64
 12  PC_5     5901 non-null   float64
 13  PC_6     5901 non-null   float64
 14  PC_7     5901 non-null   float64
 15  PC_8     5901 non-null   float64
 16  PC_9     5901 non-null   float64
 17  PC_10    5901 non-null   float64
 18  PC_11    5901 non-null   float64
 19  PC_12    5901 non-null   float64
 20  PC_13    5901 non-null   float64
 21  PC_1

In [23]:
df = raw.copy().query('freq == "month"')
df.drop(columns=["freq"], inplace=True)
df

,ipc_all,lag_1,lag_2,lag_3,lag_6,lag_9,lag_12,PC_1,PC_2,PC_3,...,PC_70,PC_71,PC_72,PC_73,PC_74,PC_75,PC_76,PC_77,PC_78,PC_79
2010-01-31,0.074472,0.263790,0.457498,0.784931,1.447299,5.323961,11.041182,-20.950868,-4.418749,-4.059225,...,-0.206838,0.640194,-0.391388,0.037969,-0.648990,-0.630158,0.242168,0.687370,0.507786,-0.185112
2010-02-28,0.312381,0.074472,0.263790,0.457498,1.417620,3.200128,8.128586,-21.291820,-4.922165,-4.142244,...,0.601718,0.185915,-0.063631,-0.348191,-0.281227,0.300365,-0.004543,-0.024545,0.327087,-0.873306
2010-03-31,0.685510,0.312381,0.074472,0.263790,0.643999,2.117642,6.563669,-22.106012,-5.376258,-4.128676,...,-0.507185,-0.140508,-0.231036,0.202919,-0.472550,-0.003362,0.032034,0.156848,0.252724,-0.745475
2010-04-30,1.215574,0.685510,0.312381,0.074472,0.784931,1.447299,5.323961,-22.300669,-5.628085,-4.900674,...,-0.054548,-0.338693,-0.184193,0.349557,0.626751,0.131864,-0.442264,0.145486,0.312705,0.330741
2010-05-31,1.384706,1.215574,0.685510,0.312381,0.457498,1.417620,3.200128,-21.607559,-5.208545,-4.925070,...,-0.566471,0.097459,0.835021,0.073013,0.048756,-0.826539,-0.660449,-0.512215,0.073288,1.420272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-30,3.844356,3.516994,3.460018,3.063085,2.118325,2.847826,2.725282,22.494737,-5.675349,-0.219020,...,0.236452,-0.208567,0.001568,0.310159,-0.377008,0.610293,-0.272084,0.991145,-0.240330,0.476591
2024-07-31,3.953707,3.844356,3.516994,3.460018,1.863150,2.075432,2.700872,22.181021,-5.765160,-0.421473,...,0.339771,1.448101,-0.772272,0.653364,0.425749,-0.757967,-0.340230,0.569572,0.405675,-0.859811
2024-08-31,5.192869,3.953707,3.844356,3.516994,2.516604,1.594145,3.058496,23.364545,-6.389616,-0.268751,...,-0.384344,0.226901,0.572090,-0.492255,0.609924,-0.212489,0.286872,-1.035240,-0.069018,0.324393
2024-09-30,6.182067,5.192869,3.953707,3.844356,3.063085,2.118325,2.847826,24.191231,-6.890628,1.461836,...,-1.373972,-0.536114,0.745231,0.057540,0.190749,0.405786,0.432674,0.494689,-0.171213,0.420452


In [24]:
isna = df.isna().sum()
isna[isna > 0]

Series([], dtype: int64)

In [25]:
df.shape

(178, 86)

## **2. Train and Val sets**

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [26]:
train_size = int(len(df) * 0.8)
train, validation = df[:train_size], df[train_size:]
print(f'Obs in train set: {train.shape[0]}; variables in train set: {train.shape[1]}')
print(f'Obs in validation set: {validation.shape[0]}; variables in validation set: {validation.shape[1]}')

Obs in train set: 142; variables in train set: 86
Obs in validation set: 36; variables in validation set: 86


In [27]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 2021-11-30 to 2024-10-31
Data columns (total 86 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ipc_all  36 non-null     float64
 1   lag_1    36 non-null     float64
 2   lag_2    36 non-null     float64
 3   lag_3    36 non-null     float64
 4   lag_6    36 non-null     float64
 5   lag_9    36 non-null     float64
 6   lag_12   36 non-null     float64
 7   PC_1     36 non-null     float64
 8   PC_2     36 non-null     float64
 9   PC_3     36 non-null     float64
 10  PC_4     36 non-null     float64
 11  PC_5     36 non-null     float64
 12  PC_6     36 non-null     float64
 13  PC_7     36 non-null     float64
 14  PC_8     36 non-null     float64
 15  PC_9     36 non-null     float64
 16  PC_10    36 non-null     float64
 17  PC_11    36 non-null     float64
 18  PC_12    36 non-null     float64
 19  PC_13    36 non-null     float64
 20  PC_14    36 non-null     float64
 21  PC_15 

In [28]:
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)
validation_scaled = pd.DataFrame(scaler.transform(validation), columns=validation.columns, index=validation.index)
#test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

X_train = train_scaled.drop('ipc_all', axis=1)
y_train = train_scaled['ipc_all']

X_validation = validation_scaled.drop('ipc_all', axis=1)
y_validation = validation_scaled['ipc_all']

#X_test = test_scaled.drop('ipc_all', axis=1)

## **3. Algorithms**

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

tscv5 = TimeSeriesSplit(n_splits=5)

### **3.1. Ridge**   

**Without tuning (Validation):**
MSE:  0.052; R2:  0.544; MAE:  0.170

In [29]:
#ridge = Ridge(random_state=0)
ridge = Ridge(alpha=0.16010644613183178, fit_intercept=False, positive=True, random_state=123) 
# Fit on training set
ridge.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
ridge_train_pred = ridge.predict(X_train)
ridge_val_pred = ridge.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ridge = mean_squared_error(y_train, ridge_train_pred)
r2_train_ridge = r2_score(y_train, ridge_train_pred)
mae_train_ridge = mean_absolute_error(y_train, ridge_train_pred)
print("Train MSE: ", mse_train_ridge)
print("Train R2: ", r2_train_ridge)
print("Train MAE: ", mae_train_ridge)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

Train MSE:  0.027493387928146436
Train R2:  0.9725066120718535
Train MAE:  0.12566187134198975
Validation MSE:  0.2606121277700468
Validation R2:  0.17451109262302267
Validation MAE:  0.42751820446868205


In [17]:
# Define parameter grid for GridSearchCV
alphas_ridge = np.logspace(-1, 2, num=500)
#alphas_ridge = np.arange(1, 100, 0.05)
param_grid = {'alpha': alphas_ridge,
              'positive': [True, False],
              'fit_intercept': [True, False],
              'random_state': [0]}

# Instantiate Ridge model
ridge = Ridge()
# Define GridSearchCV object
grid_search_ridge = GridSearchCV(ridge, param_grid, cv=tscv5, scoring='neg_mean_squared_error') 
# Fit GridSearchCV on training set
grid_search_ridge.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_ridge.best_params_)
print("Best score: ", -grid_search_ridge.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_ridge = grid_search_ridge.best_estimator_
y_val_pred = best_ridge.predict(X_validation)
# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Make predictions on test set using best model from GridSearchCV
#y_test_pred_ridge = best_ridge.predict(X_test)

Best parameter:  {'alpha': 0.16010644613183178, 'fit_intercept': False, 'positive': True, 'random_state': 0}
Best score:  0.050381388841937325
Validation MSE:  0.08529389769689197
Validation R2:  0.7298315814455989
Validation MAE:  0.25275279953249163


In [19]:
# Get the coefficients from the Ridge model
coef = ridge.coef_
# Create a dataframe of feature importances
feature_importance_ridge = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_ridge = feature_importance_ridge.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_ridge.columns = ['feat_ridge', 'imp_ridge']
# Print the feature importances
feature_importance_ridge.head(15)

,feat_ridge,imp_ridge
0,lag_1,0.715549
1,oil_bol,0.080905
2,ycorn_tj,0.077268
3,sugar_lp,0.074232
4,watermelon_po,0.059858
5,chicken_sc,0.052226
6,oil_su,0.050421
7,squash_co,0.006357
8,silver,0.003962
9,sugar_cb,0.003867


### **3.2. Lasso**
Without tuning:
Validation MSE:  0.9523622169433499
Validation R2:  -0.0030766763969758415
Validation MAE:  0.8455503308136401

In [46]:
#lasso = Lasso()
lasso = Lasso(alpha=0.1, fit_intercept=False, positive=True, random_state=123)  
# Fit on training set
lasso.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
lasso_train_pred = lasso.predict(X_train)
lasso_val_pred = lasso.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_lasso = mean_squared_error(y_train, lasso_train_pred)
r2_train_lasso = r2_score(y_train, lasso_train_pred)
mae_train_lasso = mean_absolute_error(y_train, lasso_train_pred)
print("Train MSE: ", mse_train_lasso)
print("Train R2: ", r2_train_lasso)
print("Train MAE: ", mae_train_lasso)
# Calculate Forecast metrics on validation set
mse_val_lasso = mean_squared_error(y_validation, lasso_val_pred)
r2_val_lasso = r2_score(y_validation, lasso_val_pred)
mae_val_lasso = mean_absolute_error(y_validation, lasso_val_pred)
print("Validation MSE: ", mse_val_lasso)
print("Validation R2: ", r2_val_lasso)
print("Validation MAE: ", mae_val_lasso)

Train MSE:  0.01140613732010676
Train R2:  0.9885938626798932
Train MAE:  0.09096591030089334
Validation MSE:  0.04629494785951437
Validation R2:  0.594670993655202
Validation MAE:  0.20094663332870588


In [28]:
# Define parameter grid for GridSearchCV
alphas_lasso = np.logspace(-1, 2, num=500)
#alphas_lasso = np.arange(1, 100, 0.05)
param_grid = {'alpha': alphas_lasso,
              'positive': [True, False],
              'fit_intercept': [True, False],
              'random_state': [123]}

# Instantiate lasso model
lasso = Lasso()
# Define GridSearchCV object
grid_search_lasso = GridSearchCV(lasso, param_grid, cv=tscv5, scoring='neg_mean_squared_error')  
# Fit GridSearchCV on training set
grid_search_lasso.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_lasso.best_params_)
print("Best score: ", -grid_search_lasso.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_lasso = grid_search_lasso.best_estimator_
y_val_pred = best_lasso.predict(X_validation)
# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Make predictions on test set using best model from GridSearchCV
#y_test_pred_lasso = best_lasso.predict(X_test)

Best parameter:  {'alpha': 0.1, 'fit_intercept': False, 'positive': True, 'random_state': 123}
Best score:  0.015196429729357317
Validation MSE:  0.04629494785951437
Validation R2:  0.594670993655202
Validation MAE:  0.20094663332870588


In [47]:
# Get the coefficients from the Lasso model
coef = lasso.coef_
# Create a dataframe of feature importances
feature_importance_lasso = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_lasso = feature_importance_lasso.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_lasso.columns = ['feat_lasso', 'imp_lasso']
# Print the feature importances
feature_importance_lasso.head(15)

,feat_lasso,imp_lasso
0,lag_1,0.881127
1,ufv,0.009422
2,lag_6,0.008868
3,zinc,0.000000
4,grapefruit_or,0.000000
5,quinoa_tr,0.000000
6,rice2_or,0.000000
7,platano_tr,0.000000
8,soy_po,0.000000
9,grapefruit_bol,0.000000


### **3.3. ElasticNet**
Without tuning:
Validation MSE:  0.2878895501474107
Validation R2:  0.6967799772043428
Validation MAE:  0.4602160948451647

In [ ]:
#enet = ElasticNet()
enet = ElasticNet(alpha=0.11326825671361537, l1_ratio=0.1, fit_intercept=False, positive=True)  ## cv=5
# Fit on training set
enet.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
enet_train_pred = enet.predict(X_train)
enet_val_pred = enet.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_enet = mean_squared_error(y_train, enet_train_pred, squared=True)
r2_train_enet = r2_score(y_train, enet_train_pred)
mae_train_enet = mean_absolute_error(y_train, enet_train_pred)
print("Train MSE: ", mse_train_enet)
print("Train R2: ", r2_train_enet)
print("Train MAE: ", mae_train_enet)
# Calculate Forecast metrics on validation set
mse_val_enet = mean_squared_error(y_validation, enet_val_pred, squared=True)
r2_val_enet = r2_score(y_validation, enet_val_pred)
mae_val_enet = mean_absolute_error(y_validation, enet_val_pred)
print("Validation MSE: ", mse_val_enet)
print("Validation R2: ", r2_val_enet)
print("Validation MAE: ", mae_val_enet)

Train MSE:  0.0023597541727734526
Train R2:  0.9976402458272265
Train MAE:  0.03813004212820444
Validation MSE:  0.002107268648489714
Validation R2:  0.9977805167040469
Validation MAE:  0.03561740379378739


In [ ]:
# Define parameter grid for GridSearchCV
alphas_enet = np.logspace(-1, 2, num=500)
#alphas_enet = np.arange(1, 100, 0.05)
param_grid = {'alpha': alphas_enet,
              'l1_ratio': np.arange(0.01, 0.95, 0.01),
              'positive': [True, False],
              'fit_intercept': [True, False]}

# Instantiate enet model
enet = ElasticNet()

# Define GridSearchCV object
grid_search_enet = GridSearchCV(enet, param_grid, cv=5, scoring='neg_mean_squared_error')  ## works better with 5 cv
#grid_search_enet = GridSearchCV(enet, param_grid, cv=5, scoring='r2')  ## works better with 5 cv

# Fit GridSearchCV on training set
grid_search_enet.fit(X_train, y_train)

# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_enet.best_params_)
print("Best score: ", -grid_search_enet.best_score_)

# Make predictions on validation set using best model from GridSearchCV
best_enet = grid_search_enet.best_estimator_
y_val_pred = best_enet.predict(X_validation)

# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred, squared=True)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Make predictions on test set using best model from GridSearchCV
#y_test_pred_enet = best_enet.predict(X_test)

In [ ]:
# Get the coefficients from the enet model
coef = enet.coef_
# Create a dataframe of feature importances
feature_importance_enet = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_enet = feature_importance_enet.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_enet.columns = ['feat_enet', 'imp_enet']
# Print the feature importances
feature_importance_enet.head(15)

,feat_enet,imp_enet
0,ipc_nal_l1,0.086441
1,ipc_nal_l2,0.083880
2,ipc_nal_l3,0.081695
3,ufv,0.080271
4,ipc_nal_l6,0.079604
5,ipc_nal_l12,0.076850
6,paprika_tr,0.036142
7,Precio - Tema,0.033811
8,rice2_or,0.032323
9,beef_dlp,0.024307


### **3.4. ADA**
Without tuning:
Validation MSE:  0.0036814337026600556
Validation R2:  0.9961225254245258
Validation MAE:  0.0469554086526466

In [54]:
#ada = AdaBoostRegressor()
ada = AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=6), random_state=123)
# Fit the model on training data
ada.fit(X_train, y_train)
# Make predictions on the validation set
ada_train_pred = ada.predict(X_train)
ada_val_pred = ada.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ada = mean_squared_error(y_train, ada_train_pred)
r2_train_ada = r2_score(y_train, ada_train_pred)
mae_train_ada = mean_absolute_error(y_train, ada_train_pred)
print("Train MSE: ", mse_train_ada)
print("Train R2: ", r2_train_ada)
print("Train MAE: ", mae_train_ada)
# Evaluate the model on the validation set
mse_val_ada = mean_squared_error(y_validation, ada_val_pred)
r2_val_ada = r2_score(y_validation, ada_val_pred)
mae_val_ada = mean_absolute_error(y_validation, ada_val_pred)
print("Validation MSE: ", mse_val_ada)
print("Validation R2: ", r2_val_ada)
print("Validation MAE: ", mae_val_ada)

Train MSE:  5.0765147054138336e-05
Train R2:  0.9999492348529458
Train MAE:  0.0033799273800690297
Validation MSE:  0.392655103139192
Validation R2:  -2.437837391557057
Validation MAE:  0.5293726173341485


In [57]:
# Define the AdaBoost Regressor
ada = AdaBoostRegressor()

# Define the range of hyperparameters to search over
param_grid_ada = {
    #'n_estimators': range(50, 200, 5),    ##120 was selected range(50, 200, 5)
    'learning_rate': np.logspace(-5,1, 100),
    #'loss': ['linear', 'square', 'exponential']
    'random_state': [123],
    'estimator': [#DecisionTreeRegressor(max_depth=3),
                  #DecisionTreeRegressor(max_depth=4),
                  #DecisionTreeRegressor(max_depth=5),
                  DecisionTreeRegressor(max_depth=6),
                  #DecisionTreeRegressor(max_depth=7),
                  #DecisionTreeRegressor(max_depth=8),
                  ]
}

# Tune hyperparameters using GridSearchCV with TimeSeriesSplit
grid_search_ada = GridSearchCV(estimator=ada, param_grid=param_grid_ada, cv=tscv5, scoring='r2')
#grid_search_ada = GridSearchCV(estimator=ada, param_grid=param_grid_ada, cv=tscv5, scoring='neg_mean_squared_error')
grid_search_ada.fit(X_train, y_train)

# Evaluate the model using the best hyperparameters on the test set
ada_best = AdaBoostRegressor(**grid_search_ada.best_params_)
ada_best.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = ada_best.predict(X_validation)

# Evaluate the model on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Print the best hyperparameters and the best score
print("Best parameters found: ", grid_search_ada.best_params_)
print("Lowest MSE found: ", -grid_search_ada.best_score_)

# Evaluate the model on the test set
#y_test_pred_ada = ada_best.predict(X_test)

Validation MSE:  0.3740074941755078
Validation R2:  -2.274570833078731
Validation MAE:  0.5144651864672598
Best parameters found:  {'estimator': DecisionTreeRegressor(max_depth=6), 'learning_rate': 0.002009233002565048, 'random_state': 123}
Lowest MSE found:  7.190820165324213


In [ ]:
# Create a DataFrame with the feature importance values
feature_importance_ada = pd.DataFrame({'Feature': X_train.columns, 'Importance': ada.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_ada = feature_importance_ada.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_ada.columns = ['feat', 'imp_ada']
# Print the feature importance DataFrame
feature_importance_ada.head(15)

,feat,imp_ada
0,ipc_nal_l3,0.090550
1,ufv,0.084265
2,ipc_nal_l1,0.071661
3,ipc_nal_l2,0.066128
4,ipc_nal_l6,0.062670
5,flour_tj,0.062154
6,ipc_nal_l12,0.045471
7,milk2_or,0.036227
8,milk2_lp,0.036167
9,milk2_po,0.033045


### **3.5. GBR**
Without tuning:
Validation MSE:  0.0014618738110729937
Validation R2:  0.998460279610389
Validation MAE:  0.02910552415848012

In [30]:
gbr = GradientBoostingRegressor(random_state=0)
#gbr = GradientBoostingRegressor(learning_rate=0.040949150623804255, n_estimators=290, random_state=0)
# Fit the model on the training set
gbr.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
gbr_train_pred = gbr.predict(X_train)
gbr_val_pred = gbr.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_gbr = mean_squared_error(y_train, gbr_train_pred)
r2_train_gbr = r2_score(y_train, gbr_train_pred)
mae_train_gbr = mean_absolute_error(y_train, gbr_train_pred)
print("Train MSE: ", mse_train_gbr)
print("Train R2: ", r2_train_gbr)
print("Train MAE: ", mae_train_gbr)
# Calculate Forecast metrics on validation set
mse_val_gbr = mean_squared_error(y_validation, gbr_val_pred)
r2_val_gbr = r2_score(y_validation, gbr_val_pred)
mae_val_gbr = mean_absolute_error(y_validation, gbr_val_pred)
print("Validation MSE: ", mse_val_gbr)
print("Validation R2: ", r2_val_gbr)
print("Validation MAE: ", mae_val_gbr)

Train MSE:  0.00015644566674301836
Train R2:  0.999843554333257
Train MAE:  0.010221580621379045
Validation MSE:  0.03796388849103872
Validation R2:  0.8797493842730874
Validation MAE:  0.15234983069993324


In [32]:
# Define the model
gbr = GradientBoostingRegressor()

# Define the hyperparameters to be tuned
params = {
    'learning_rate': np.logspace(-2,0.5, 50),
    #'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],   ## 'loss': 'squared_error'
    'n_estimators': range(100, 300, 10),
    'max_depth': range(3,6,1),
    #'min_samples_split': range(2,20,1),
    #'min_samples_leaf': range(1,50, 1),
    #'min_weight_fraction_leaf': np.arange(0.0, 0.5, 0.01),
    #'subsample': np.arange(0.5, 1, 0.05),
    #'max_features': [None, 'sqrt', 'log2'],
    #'max_leaf_nodes': range(2, 200, 1),
    #'criterion': ['friedman_mse', 'squared_error'],
    'random_state': [123]
}

# Create the GridSearchCV object
#grid_gbr = GridSearchCV(gbr, params, cv=tscv5, scoring='r2', n_jobs=-1)
grid_gbr = GridSearchCV(gbr, params, cv=tscv5, scoring='neg_mean_squared_error', n_jobs=-1)
# Fit the model on the training set with GridSearchCV
grid_gbr.fit(X_train, y_train)
# Print the best hyperparameters
print('Best hyperparameters:', grid_gbr.best_params_)
# Use the best model to make predictions on the validation set
y_val_pred = grid_gbr.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Use the best model to make predictions on the test set
#y_test_pred_gbr = grid_gbr.predict(X_test)

c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:1114: RuntimeWarning: overflow encountered in square
  (array - array_means[:, np.newaxis]) ** 2, axis=1, weights=weights
c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


Best hyperparameters: {'learning_rate': 1.757510624854793, 'max_depth': 5, 'n_estimators': 100, 'random_state': 123}
Validation MSE:  0.2815678564215272
Validation R2:  -1.4652283831985624
Validation MAE:  0.37399708361903783


In [33]:
# Create a DataFrame with the feature importance values
feature_importance_gbr = pd.DataFrame({'Feature': X_train.columns, 'Importance': gbr.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_gbr = feature_importance_gbr.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_gbr.columns = ['feat_gbr', 'imp_gbr']
# Print the feature importance DataFrame
feature_importance_gbr.head(15)

NotFittedError: This GradientBoostingRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

### **3.4. RF**
Without tuning:
Validation MSE:  0.00100147205985504
Validation R2:  0.9989451983211514
Validation MAE:  0.026535714623290185

In [31]:
# Define the Random Forest Regression model
rf = RandomForestRegressor(random_state=0)
#rf = RandomForestRegressor(min_samples_split=2, n_estimators=265, random_state=0)

# Fit the model to the training data and make predictions on the validation set
rf.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
rf_train_pred = rf.predict(X_train)
rf_val_pred = rf.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_rf = mean_squared_error(y_train, rf_train_pred)
r2_train_rf = r2_score(y_train, rf_train_pred)
mae_train_rf = mean_absolute_error(y_train, rf_train_pred)
print("Train MSE: ", mse_train_rf)
print("Train R2: ", r2_train_rf)
print("Train MAE: ", mae_train_rf)
# Calculate Forecast metrics on validation set
mse_val_rf = mean_squared_error(y_validation, rf_val_pred)
r2_val_rf = r2_score(y_validation, rf_val_pred)
mae_val_rf = mean_absolute_error(y_validation, rf_val_pred)
print("Validation MSE: ", mse_val_rf)
print("Validation R2: ", r2_val_rf)
print("Validation MAE: ", mae_val_rf)

Train MSE:  0.008122951055042425
Train R2:  0.9918770489449575
Train MAE:  0.06814372849503035
Validation MSE:  0.04705790793228797
Validation R2:  0.8509440779488716
Validation MAE:  0.1665860005700532


In [ ]:
# Define the Random Forest Regression model
rf_reg = RandomForestRegressor()

# Define the hyperparameters to tune
param_grid_rf = {
    'n_estimators': range(100, 300, 15),
    #'max_features': [None, 'sqrt', 'sqrt']
    #'max_depth': range(3,7,1),
    'min_samples_split': range(2,20,1),
    #'min_samples_leaf': range(1,50, 1),
    #'min_weight_fraction_leaf': np.arange(0.0, 0.5, 0.01),
    #'bootstrap': [True],
    #'oob_score': [True, False],
    #'warm_start': [True, False],
    #'max_samples': np.arange(0.1, 1.0, 0.01)
    'random_state': [0]
}

# Define the GridSearchCV object
#grid_rf_reg = GridSearchCV(estimator=rf_reg, param_grid=param_grid_rf, cv=5, scoring='r2')
grid_rf_reg = GridSearchCV(estimator=rf_reg, param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_rf_reg.fit(X_train, y_train)

# Extract the best hyperparameters and score
best_params = grid_rf_reg.best_params_
best_score = grid_rf_reg.best_score_

# Print the best hyperparameters found by GridSearchCV
print(f"Best hyperparameters: {best_params}")
print(f"Best score: {best_score}")

# Instantiate a new Random Forest Regression model using the best hyperparameters
rf_reg_best = RandomForestRegressor(**best_params)

# Fit the model to the training data and make predictions on the validation set
rf_reg_best.fit(X_train, y_train)

# Use the best model to make predictions on the validation set
y_val_pred = rf_reg_best.predict(X_validation)

# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_rf = rf_reg_best.predict(X_test)

Best hyperparameters: {'min_samples_split': 2, 'n_estimators': 265, 'random_state': 0}
Best score: -0.0035817383690906874
Validation MSE:  0.0009983900502684609
Validation R2:  0.9989484444515393
Validation MAE:  0.024970310604635504


In [ ]:
# Create a DataFrame with the feature importance values
feature_importance_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_rf = feature_importance_rf.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_rf.columns = ['feat', 'imp_rf']
# Print the feature importance DataFrame
feature_importance_rf.head(30)

,feat,imp_rf
0,ipc_nal_l6,0.137714
1,ipc_nal_l2,0.118994
2,ufv,0.117774
3,ipc_nal_l12,0.110887
4,ipc_nal_l1,0.104852
5,ipc_nal_l3,0.094636
6,flour_tj,0.090228
7,milk2_po,0.023239
8,milk2_or,0.023221
9,milk_bol,0.015986


### **3.5. ET**

In [61]:
# Define the Extra Trees Regression model
et = ExtraTreesRegressor()
#et = ExtraTreesRegressor(bootstrap=True, max_samples=0.9599999999999995, oob_score=True)
# Fit the model to the training data and make predictions on the validation set
et.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
et_train_pred = et.predict(X_train)
et_val_pred = et.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_et = mean_squared_error(y_train, et_train_pred)
r2_train_et = r2_score(y_train, et_train_pred)
mae_train_et = mean_absolute_error(y_train, et_train_pred)
print("Train MSE: ", mse_train_et)
print("Train R2: ", r2_train_et)
print("Train MAE: ", mae_train_et)
# Calculate Forecast metrics on validation set
mse_val_et = mean_squared_error(y_validation, et_val_pred)
r2_val_et = r2_score(y_validation, et_val_pred)
mae_val_et = mean_absolute_error(y_validation, et_val_pred)
print("Validation MSE: ", mse_val_et)
print("Validation R2: ", r2_val_et)
print("Validation MAE: ", mae_val_et)

Train MSE:  1.926646172382771e-30
Train R2:  1.0
Train MAE:  1.0490214547189142e-15
Validation MSE:  0.4424822396169083
Validation R2:  -2.8740919863091
Validation MAE:  0.5724749248973184


In [ ]:
# Define the Extra Trees Regression model
et_reg = ExtraTreesRegressor()

# Define the hyperparameter grid to search over
param_grid = {
    #'n_estimators': range(100, 300, 15),
    #'max_depth': range(3,15,1),
    #'max_features': [None, 'sqrt', 'sqrt']
    'min_samples_split': range(2,20,1),
    'min_samples_leaf': range(1,50, 1),
    #'min_weight_fraction_leaf': np.arange(0.0, 0.5, 0.01),
    'bootstrap': [True],
    'oob_score': [True, False],
    #'warm_start': [True, False],
    'max_samples': np.arange(0.1, 1.0, 0.01)
    #'criterion': ['squared_error', 'absolute_error', 'friedman_mse'],
    #'min_impurity_decrease': np.arange(0.0, 0.01, 0.00001),
    #'random_state': [0]
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(et_reg, param_grid=param_grid, cv=5, scoring='r2')
#grid_search = GridSearchCV(et_reg, param_grid=param_grid, cv=tscv5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding validation score
print("Best hyperparameters: ", grid_search.best_params_)
#print("Validation score: ", grid_search.best_score_)

# Use the best model to make predictions on the validation set
best_et_reg = grid_search.best_estimator_
y_val_pred = best_et_reg.predict(X_validation)

# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_et = best_et_reg.predict(X_test)

In [ ]:
# Create a DataFrame with the feature importance values
feature_importance_et = pd.DataFrame({'Feature': X_train.columns, 'Importance': et.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_et = feature_importance_et.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_et.columns = ['feat', 'imp_et']
# Print the feature importance DataFrame
feature_importance_et.head(30)

,feat,imp_et
0,ycorn_ea,0.128451
1,ipc_nal_l1,0.115000
2,ufv,0.081138
3,ipc_nal_l2,0.073717
4,ipc_nal_l12,0.073355
5,flour_tj,0.068439
6,ipc_nal_l6,0.058653
7,milk_dlp,0.053411
8,milk_lp,0.047272
9,milk_bol,0.045265


## **4. Report**

#### End